In [1]:
  // обрезать после домена ?


import org.apache.spark.sql._
import org.apache.spark.ml._
import java.io._
import play.api.libs.json._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
import java.net.URLDecoder.decode

In [2]:
val vehicleUsers = spark.read.json("/labs/laba02/autousers.json").toDF()

vehicleUsers = [autousers: array<string>]


[autousers: array<string>]

In [3]:
val logs = spark.read.option("delimiter", "\t").textFile("/labs/laba02/logs")

logs = [value: string]


[value: string]

In [4]:
logs.show(3, truncate=false)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251	1423724290.637                                                                          |
|182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [5]:
// split df to array rows

val col_ = split(col("value"), "\t").alias("value")
val logs_splited = logs.select(col_)
logs_splited.show(3, truncate=false)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|[258131083251, 1423724291.637, http%3A%2F%2Fwww.zakon.kz%2F]                                             |
|[258131083251, 1423724290.637]                                                                           |
|[182984926014, 1423724291.666, http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5]|
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



col_ = split(value, 	) AS `value`
logs_splited = [value: array<string>]


[value: array<string>]

In [6]:
// selecting columns by index of arrays

val logs_df = logs_splited.withColumn("UID", $"value"(0))
                    .withColumn("Timestamp", $"value"(1))
                    .withColumn("URL", $"value"(2))
                    .withColumn("Timestamp", to_timestamp(from_unixtime($"Timestamp")))
                    .select("UID","Timestamp", "URL")
                    .filter($"URL".like("%http%"))
//                    .filter($"URL".like("%http_://%"))


logs_df = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [7]:
// logs_df.select($"URL").count()

Name: Syntax Error.
Message: 
StackTrace: 

In [8]:
val logs_df_replaced = logs_df.withColumn("URL", regexp_replace($"URL", lit("u0"), lit("")))withColumn("URL", regexp_replace($"URL", lit("u2"), lit("")))

logs_df_replaced = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [9]:
// logs_df_replaced.select($"URL").filter($"URL".like("%u[02]%")).count()

Name: Syntax Error.
Message: 
StackTrace: 

In [10]:
// encoding example
"""val url = "http%3A%2F%2Finixi.ru%2Fc246-7497.html"
val s = java.net.URLDecoder.decode(url, "UTF-8")"""

val url = "http%3A%2F%2Finixi.ru%2Fc246-7497.html"
val s = java.net.URLDecoder.decode(url, "UTF-8")


In [11]:
import org.apache.spark.sql.functions.{col, udf}
val urlDecoder = udf((s: String) => java.net.URLDecoder.decode(s, "UTF-8"))


val logs_df_udf = logs_df_replaced.filter($"URL".like("%http%")).withColumn("URL", urlDecoder(col("URL")) as "URL")
logs_df_udf.show(truncate=false)

+------------+-------------------+-------------------------------------------------------+
|UID         |Timestamp          |URL                                                    |
+------------+-------------------+-------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http://www.zakon.kz/                                   |
|182984926014|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=5|
|182984926014|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=4|
|289380960531|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/forumdisplay.php?f=128        |
|289380960531|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/                              |
|85356233460 |2015-02-12 09:58:10|http://www.wmmail.ru/index.php?cf=usf-userfing         |
|204109491902|2015-02-12 09:58:11|https://vk.com/id21743084                              |
|204109491902|2015-02-12 09:58:10|https://vk.com/feed?section=updates                    |

urlDecoder = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
logs_df_udf = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [12]:
val logs_df_domen = logs_df_udf.filter(($"URL".like("%http://%")) || ($"URL".like("%https://%")))
logs_df_domen.show(5, false)


+------------+-------------------+-------------------------------------------------------+
|UID         |Timestamp          |URL                                                    |
+------------+-------------------+-------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http://www.zakon.kz/                                   |
|182984926014|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=5|
|182984926014|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/showthread.php?t=875590&page=4|
|289380960531|2015-02-12 09:58:11|http://www.bmwclub.ru/vb/forumdisplay.php?f=128        |
|289380960531|2015-02-12 09:58:10|http://www.bmwclub.ru/vb/                              |
+------------+-------------------+-------------------------------------------------------+
only showing top 5 rows



logs_df_domen = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [13]:
// extracting neat URL

val logs_decoded = logs_df_domen.withColumn("URL", lower(callUDF("parse_url", $"URL", lit("HOST"))))
                               .withColumn("URL", regexp_replace(lower($"URL"), "www.", ""))
logs_decoded.show(5)

+------------+-------------------+----------+
|         UID|          Timestamp|       URL|
+------------+-------------------+----------+
|258131083251|2015-02-12 09:58:11|  zakon.kz|
|182984926014|2015-02-12 09:58:11|bmwclub.ru|
|182984926014|2015-02-12 09:58:10|bmwclub.ru|
|289380960531|2015-02-12 09:58:11|bmwclub.ru|
|289380960531|2015-02-12 09:58:10|bmwclub.ru|
+------------+-------------------+----------+
only showing top 5 rows



logs_decoded = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [14]:
// extracting neat URL
"""
val substrDel = substring_index($"URL", ".", 1)
val logs_df_clean = logs_df.withColumn("URL", substring_index(substring(regexp_replace($"URL",substrDel , lit("")), 2, 999), "%", 1))

logs_df_clean.show(numRows=5, truncate=200 /*, vertical=true*/)"""

"
val substrDel = substring_index($"URL", ".", 1)
val logs_df_clean = logs_df.withColumn("URL", substring_index(substring(regexp_replace($"URL",substrDel , lit("")), 2, 999), "%", 1))
logs_df_clean.show(numRows=5, truncate=200 /*, vertical=true*/)"


In [15]:
logs_decoded.printSchema()

root
 |-- UID: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- URL: string (nullable = true)



In [16]:
val vehicleUsers_df = vehicleUsers.select(explode($"autousers").alias("autousers"))
vehicleUsers_df.show(3)

+------------+
|   autousers|
+------------+
|100341861572|
|100473724387|
|100528753939|
+------------+
only showing top 3 rows



vehicleUsers_df = [autousers: string]


[autousers: string]

In [17]:
logs_decoded.select($"UID").distinct.count()

100000

In [18]:
vehicleUsers_df.select("autousers").distinct().count()

5209

In [19]:
val joined_df = logs_decoded.join(vehicleUsers_df, $"UID" === $"autousers", "left")
joined_df.show()
joined_df.count()

+------------+-------------------+-----------------+------------+
|         UID|          Timestamp|              URL|   autousers|
+------------+-------------------+-----------------+------------+
|258131083251|2015-02-12 09:58:11|         zakon.kz|        null|
|182984926014|2015-02-12 09:58:11|       bmwclub.ru|182984926014|
|182984926014|2015-02-12 09:58:10|       bmwclub.ru|182984926014|
|289380960531|2015-02-12 09:58:11|       bmwclub.ru|289380960531|
|289380960531|2015-02-12 09:58:10|       bmwclub.ru|289380960531|
| 85356233460|2015-02-12 09:58:10|        wmmail.ru|        null|
|204109491902|2015-02-12 09:58:11|           vk.com|        null|
|204109491902|2015-02-12 09:58:10|           vk.com|        null|
|302580370974|2015-02-12 09:58:11|  novayagazeta.ru|        null|
|160372190500|2015-02-12 09:58:11|forum.krasmama.ru|        null|
|160372190500|2015-02-12 09:58:10|forum.krasmama.ru|        null|
|310139244463|2015-02-12 09:58:12|       smotri.com|        null|
|310139244

joined_df = [UID: string, Timestamp: timestamp ... 2 more fields]


6571047

In [51]:
joined_df.orderBy($"UID".desc).show(10)
joined_df.orderBy($"UID".asc).show(10)

+-----------+-------------------+----------+-----------+
|        UID|          Timestamp|       URL|  autousers|
+-----------+-------------------+----------+-----------+
|99751967102|2015-02-12 15:23:02|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:16|bmwclub.ru|99751967102|
|99751967102|2015-02-12 15:17:12|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:25:10|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:29|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:30|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:17|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:34|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:23:33|bmwclub.ru|99751967102|
|99751967102|2015-02-12 16:25:09|bmwclub.ru|99751967102|
+-----------+-------------------+----------+-----------+
only showing top 10 rows

+------------+-------------------+--------------+------------+
|         UID|          Timestamp|           URL|   autousers|
+------------+-------------------+--------------+-

In [20]:
// to int val joined_df_int = 
val joined_df_int = joined_df.withColumn("UID", $"UID".cast("long"))

joined_df_int = [UID: bigint, Timestamp: timestamp ... 2 more fields]


[UID: bigint, Timestamp: timestamp ... 2 more fields]

In [62]:
val joined_df_two_cols = joined_df_int.withColumn("Auto-flag", when($"autousers".isNotNull, 1).otherwise(0))
                                .select($"URL", $"Auto-flag")
    joined_df_two_cols.show()         

+-----------------+---------+
|              URL|Auto-flag|
+-----------------+---------+
|         zakon.kz|        0|
|       bmwclub.ru|        1|
|       bmwclub.ru|        1|
|       bmwclub.ru|        1|
|       bmwclub.ru|        1|
|        wmmail.ru|        0|
|           vk.com|        0|
|           vk.com|        0|
|  novayagazeta.ru|        0|
|forum.krasmama.ru|        0|
|forum.krasmama.ru|        0|
|       smotri.com|        0|
|       smotri.com|        0|
|  psychologies.ru|        0|
|      echo.msk.ru|        0|
|      echo.msk.ru|        0|
|        wmmail.ru|        0|
|      echo.msk.ru|        0|
|      echo.msk.ru|        0|
|       smotri.com|        0|
+-----------------+---------+
only showing top 20 rows



joined_df_two_cols = [URL: string, Auto-flag: int]


[URL: string, Auto-flag: int]

In [29]:
import org.apache.spark.sql.expressions.Window

val window = Window.partitionBy("URL")

window = org.apache.spark.sql.expressions.WindowSpec@4fe2e31d


org.apache.spark.sql.expressions.WindowSpec@4fe2e31d

In [66]:
val laba02_domains = joined_df_two_cols.withColumn("P_domain_and_driver_sqrt", sum($"Auto-flag").over(window))
                          .withColumn("P_domain_and_driver_sqrt", $"P_domain_and_driver_sqrt" * $"P_domain_and_driver_sqrt")
                          .withColumn("P_domain", count($"URL").over(window))
                          .withColumn("P_driver", sum($"Auto-flag").over(Window.partitionBy()))
                          .withColumn("relevance", round($"P_domain_and_driver_sqrt" / ($"P_domain" * $"P_driver"), 4))
                          .select($"URL".as("domain"), $"relevance")
                          .orderBy($"relevance".desc)

laba02_domains = [domain: string, relevance: double]


[domain: string, relevance: double]

In [67]:
laba02_domains.distinct.show()

+-------------------+---------+
|             domain|relevance|
+-------------------+---------+
|     avto-russia.ru|   0.3593|
|         bmwclub.ru|     0.31|
|            cars.ru|   0.2065|
|     auto.yandex.ru|   0.0026|
|       passat-b5.ru|   0.0026|
|             as8.ru|   0.0015|
|      avtogermes.ru|   0.0013|
|         club-fx.ru|   0.0011|
|      spravka003.ru|    0.001|
|       car-total.ru|   9.0E-4|
|         e36club.ru|   7.0E-4|
|           eleks.ru|   6.0E-4|
|          yandex.ru|   6.0E-4|
|stonerrock.borda.ru|   6.0E-4|
|        e38-club.ru|   6.0E-4|
| shop.kalyan-hut.ru|   6.0E-4|
|            vwts.ru|   5.0E-4|
| forums.warforge.ru|   5.0E-4|
|     se7enkills.net|   4.0E-4|
|           inixi.ru|   4.0E-4|
+-------------------+---------+
only showing top 20 rows



### testing data

In [22]:
val someDF = Seq(
  ("cars.ru", 1),
  ("auto.ru", 1),
  ("ya.ru", 1),
    ("ya.ru", 0),
    ("auto.ru", 0),
    ("ya.ru", 0),
    ("market.ya.ru", 0),
    ("auto.ru", 1),
    ("cars.ru", 1),
    ("ya.ru", 1)
).toDF("URL", "Auto-flag")

someDF = [URL: string, Auto-flag: int]


[URL: string, Auto-flag: int]

In [23]:
someDF.show()

+------------+---------+
|         URL|Auto-flag|
+------------+---------+
|     cars.ru|        1|
|     auto.ru|        1|
|       ya.ru|        1|
|       ya.ru|        0|
|     auto.ru|        0|
|       ya.ru|        0|
|market.ya.ru|        0|
|     auto.ru|        1|
|     cars.ru|        1|
|       ya.ru|        1|
+------------+---------+



In [56]:
someDF.withColumn("P_domain_and_driver_sqrt", sum($"Auto-flag").over(window))
      .withColumn("P_domain_and_driver_sqrt", $"P_domain_and_driver_sqrt" * $"P_domain_and_driver_sqrt")
      .withColumn("P_domain", count($"URL").over(window))
      .withColumn("P_driver", sum($"Auto-flag").over(Window.partitionBy()))
      .withColumn("relevance", round($"P_domain_and_driver_sqrt" / ($"P_domain" * $"P_driver"), 4))
      .select($"URL", $"relevance")
      .orderBy($"relevance".desc).show()


+------------+---------+
|         URL|relevance|
+------------+---------+
|     cars.ru|   0.3333|
|     cars.ru|   0.3333|
|     auto.ru|   0.2222|
|     auto.ru|   0.2222|
|     auto.ru|   0.2222|
|       ya.ru|   0.1667|
|       ya.ru|   0.1667|
|       ya.ru|   0.1667|
|       ya.ru|   0.1667|
|market.ya.ru|      0.0|
+------------+---------+

